# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm#MID), thank goodness we can search for these things.

## Preparation: Knowing your tags

These questions are the same for every data set, and might not work exactly for yours.

**Search for every operator with 'dirt' in their name, including abandoned mines.**

### What is the tag and class name for every row of data?

In [1]:
#tr  

In [ ]:
#<td>

In [3]:
#<align = "center">

### What is the tag and class name for every mine operator's name?

In [ ]:
#td  

### What is the tag and class name for every mine's name?

In [ ]:
# <font style="FONT-SIZE:.75em;">

### What is the tag and class name for every mine operator's name?

In [ ]:
#<td> 

### What is the tag and class name for every mine operator's name?

In [ ]:
#td

## Being lazy

If you only needed these results, what would you do instead of scraping them?

In [ ]:
#put it in excel 

## Setup: Import what you'll need to scrape the page

Use `requests`, not `urllib`.

In [5]:
from bs4 import BeautifulSoup
import requests

In [14]:
url = 'https://arlweb.msha.gov/drs/ASP/OprNameStatesearch.asp'
data = {

    'OperSearch':'dirt',
    'Abandoned':'No',
    'MineName':'',
    'StateSearch':'None',
    'CM':'All',
    'x':'23',
    'y':'1',
    'MC':'Opersearch'
}

response = requests.post(url, data = data)
doc = BeautifulSoup(response.text, 'html.parser')
doc

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">

<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<!-- ****************************************** Begin META TAGS ********************************************* -->
<meta content="NOINDEX, NOFOLLOW" name="ROBOTS">
<!-- ****************************************** End META TAGS *********************************************** -->
<title>MSHA  - Mine  Data Retrieval System - Basic Mine Information Page</title>
<script src="/2010redesign/Scripts/federated-analytics.js" type="text/javascript"></script>
<script src="/2010redesign/Scripts/AC_RunActiveContent.js" type="text/javascript"></script>
<link href="/2010Redesign/includes/Print.css" media="print" rel="stylesheet" type="text/css">
<link href="/2010Redesign/Includes/MSHAwebnew.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="/2010Redesign/includes/style-screen.css" media="screen" rel="stylesheet" type="text/

## Try to scrape the page

To test if you requested the page correctly, save the BeautifulSoup document as `doc` and run the code `doc.find_all('tr')[-1].text` to get the text of the last `<tr>` element.

- If the result starts with **Total Number of Mines Found**, you were successful.

In [15]:
doc.find_all('tr')[-1].text

'\nTotal Number of Mines Found:\xa0\xa019'

## Actually scraping

### Hopefully you know that each `tr` is supposed to be your data. What is the index of the first row element that is actually a result?

`.text` will help you here.

In [17]:
doc.find('tr').text

'\n\nOperator Name or Mine Name Search\n\xa0'

In [31]:
trs = doc.find_all('tr')

In [37]:
tds = doc.find_all('td')
tds

[<td width="30%"><a href="/drs/drshome.htm"><img alt="Mine Data Retrieval System" border="0" height="75" src="/drs/images/drslogo.png" width="300"/></a></td>,
 <td width="30%"> </td>,
 <td valign="top" width="50%">
 <table width="100%">
 <tr>
 <td><font style="FONT-SIZE:.80em;"><b>Abandoned*</b></font></td></tr>
 <tr>
 <td valign="top" width="95%"><font style="FONT-SIZE:.75em;">Indicates Mine is Abandoned and Sealed</font></td></tr></table></td>,
 <td><font style="FONT-SIZE:.80em;"><b>Abandoned*</b></font></td>,
 <td valign="top" width="95%"><font style="FONT-SIZE:.75em;">Indicates Mine is Abandoned and Sealed</font></td>,
 <td align="right" valign="top" width="50%">
 <table align="right" width="100%">
 <tr>
 <td align="right" colspan="2"><font style="FONT-SIZE:.80em;"><b>*CM (Coal or Metal Mine/Nonmetal Mine)</b></font></td></tr>
 <tr>
 <td align="right" width="46%"><font style="FONT-SIZE:.80em;">C<br>M</br></font></td>
 <td width="54%"><font style="FONT-SIZE:.80em;">...... Coal<br>..

In [36]:
doc.find_all('td')[4]

<td valign="top" width="95%"><font style="FONT-SIZE:.75em;">Indicates Mine is Abandoned and Sealed</font></td>

### Loop through each operator result, printing its name

Use LIST SLICING to skip the non-data row(s).

In [67]:
for row in trs:
    if len(row.find_all('td')) > 7:
        name = row.find_all('td')[2].text
        print(name)

 Newberg Rock & Dirt  
AM Dirtworks & Aggregate Sales  
Dirt Company  
Dirt Con  
Dirt Doctor Inc  
Dirt Works  
Holley Dirt Company, Inc  
Krueger Brothers Gravel & Dirt  
M R Dirt  
M.R. Dirt Inc.  
P B Dirt Movers, Inc  
PB Dirt Movers  
PB Dirt Movers, Inc  
Prescott Dirt, LLC  
R D Blankenship Dirt Work LLC  
SIMPSON DIRTWORX LLC  
SIMPSON DIRTWORX LLC  
Spry's Dirt & Gravel, Inc.  
Vogt Dirt Service  


### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [82]:
for row in trs:
    if len(row.find_all('td')) > 7:
        name = row.find_all('td')[0].text.strip()
        print(name)
        print('_____')

3503598
_____
4801789
_____
5001797
_____
4608254
_____
2103723
_____
4104757
_____
0801306
_____
3901432
_____
3609624
_____
3609931
_____
1519799
_____
4407296
_____
4407270
_____
0203332
_____
2901986
_____
4300768
_____
4300776
_____
2302283
_____
2103518
_____


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

In [81]:
mines =[]
for row in trs[7:-2]:
    current = {}
    ID = row.find_all('td')[0]
    if ID:
        current['OperatorID'] = ID.text.strip()
    name = row.find_all('td')[2]
    if name:
        current['Name'] = name.text.strip()
    mine_name = row.find_all('td')[3]
    if mine_name:
        current['Mine_name'] = mine_name.text.strip()
    state = row.find_all('td')[1]
    if state:
        current['State'] = state.text.strip()
    mine_type = row.find_all('td')[4]
    if mine_type:
        current['Mine_type'] = mine_type.text.strip()
    coal = row.find_all('td')[5]
    if coal:
        current['Coal_or_Metal'] = coal.text.strip()
    status = row.find_all('td')[6]
    if status:
        current['Status'] = status.text.strip()
    commodity = row.find_all('td')[7]
    if commodity:
        current['Commodity'] = commodity.text.strip()
    mines.append(current)
    print(current)
    print("---")

{'OperatorID': '3503598', 'Name': 'Newberg Rock & Dirt', 'Mine_name': 'Newberg Rock & Dirt', 'State': 'OR', 'Mine_type': 'Surface', 'Coal_or_Metal': 'M', 'Status': 'Active', 'Commodity': 'Crushed, Broken Stone NEC'}
---
{'OperatorID': '4801789', 'Name': 'AM Dirtworks & Aggregate Sales', 'Mine_name': 'AM Dirtworks & Aggregate Sales', 'State': 'ND', 'Mine_type': 'Surface', 'Coal_or_Metal': 'M', 'Status': 'Intermittent', 'Commodity': 'Construction Sand and Gravel'}
---
{'OperatorID': '5001797', 'Name': 'Dirt Company', 'Mine_name': 'Bush Pilot', 'State': 'AK', 'Mine_type': 'Surface', 'Coal_or_Metal': 'M', 'Status': 'Intermittent', 'Commodity': 'Construction Sand and Gravel'}
---
{'OperatorID': '4608254', 'Name': 'Dirt Con', 'Mine_name': 'Hog Lick Quarry', 'State': 'WV', 'Mine_type': 'Surface', 'Coal_or_Metal': 'M', 'Status': 'Temporarily Idled', 'Commodity': 'Crushed, Broken Limestone NEC'}
---
{'OperatorID': '2103723', 'Name': 'Dirt Doctor Inc', 'Mine_name': 'Rock Lake Plant', 'State': 'M

### Save that to a CSV

In [83]:
import pandas as pd
df = pd.DataFrame(mines)

In [84]:
df.to_csv("mines.csv", index = False)

In [89]:
mines_df = pd.read_csv("mines.csv")
df.dtypes

Coal_or_Metal    object
Commodity        object
Mine_name        object
Mine_type        object
Name             object
OperatorID       object
State            object
Status           object
dtype: object

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [90]:
mines_df

,Coal_or_Metal,Commodity,Mine_name,Mine_type,Name,OperatorID,State,Status
0,M,"Crushed, Broken Stone NEC",Newberg Rock & Dirt,Surface,Newberg Rock & Dirt,3503598,OR,Active
1,M,Construction Sand and Gravel,AM Dirtworks & Aggregate Sales,Surface,AM Dirtworks & Aggregate Sales,4801789,ND,Intermittent
2,M,Construction Sand and Gravel,Bush Pilot,Surface,Dirt Company,5001797,AK,Intermittent
3,M,"Crushed, Broken Limestone NEC",Hog Lick Quarry,Surface,Dirt Con,4608254,WV,Temporarily Idled
4,M,Construction Sand and Gravel,Rock Lake Plant,Surface,Dirt Doctor Inc,2103723,MN,Intermittent
5,M,Construction Sand and Gravel,Portable #1,Surface,Dirt Works,4104757,TX,Intermittent
6,M,"Sand, Common",River Road Pit,Surface,"Holley Dirt Company, Inc",801306,FL,Active
7,M,Construction Sand and Gravel,PORTABLE SCREENER,Surface,Krueger Brothers Gravel & Dirt,3901432,SD,Intermittent
8,M,Construction Sand and Gravel,Forbes Pit,Surface,M R Dirt,3609624,PA,Intermittent
9,M,Dimension Stone NEC,Camptown Quarry,Surface,M.R. Dirt Inc.,3609931,PA,Intermittent
